In [33]:
import pandas as pd
import numpy as np

# 50557 위키약사님 주문 내역 데이터 다운로드
df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_50557_2024-02-28~2025-02-28.xlsx')

# 결제완료인 건들만 필터링
con1 = df.iloc[:, 4] == '배송중'
con2 = df.iloc[:, 4] == '배송완료'
con3 = df.iloc[:, 4] == '결제완료'
con4 = df.iloc[:, 4] == '구매확정'
df = df[con1 | con2 | con3 | con4]

# 중요한 컬럼만 남기기
df = df.iloc[:, [0, 3, 6, 7, 9, 13]]

# 컬럼명 영문으로 바꾸기
pre_col = df.columns.tolist()
post_col = ['order_no', 'order_dt', 'option_name', 'quantity', 'price', 'revenue']
cols = dict(zip(pre_col, post_col))
df.rename(columns=cols, inplace=True)
df.reset_index(drop=True, inplace=True)

# order_date 만들기
df['order_date'] = df['order_dt'].dt.strftime('%Y-%m-%d')

# order_date 별로 그룹바이 해서 df2 만들기
df2 = pd.DataFrame(df.groupby('order_date')['option_name'].value_counts())
df2 = df2.reset_index()

# 옵션별 통계
df2['box_count'] = 0
df2['revenue'] = 0

for i in df2['option_name'].unique():
    if i == '6박스(6개월분)_35%':
        con = df2['option_name'] == i
        df2.loc[con, 'box_count'] = 6
        df2.loc[con, 'revenue'] = df2.loc[con, 'count'] * 156000
    elif i == '4박스(4개월분)_28%':
        con = df2['option_name'] == i
        df2.loc[con, 'box_count'] = 4
        df2.loc[con, 'revenue'] = df2.loc[con, 'count'] * 115200
    elif i == '12박스(12개월분)_45%':
        con = df2['option_name'] == i
        df2.loc[con, 'box_count'] = 12
        df2.loc[con, 'revenue'] = df2.loc[con, 'count'] * 264000

df2['box_count'] = df2['box_count'] * df2['count']

# 컬럼명 다시 한글로 바꾸기
col_before = df2.columns.tolist()
col_after = ['주문일', '옵션명', '주문수량', '박스수량', '판매액']
cols = dict(zip(col_before, col_after))
df2.rename(columns=cols, inplace=True)

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [35]:
df2

,주문일,옵션명,주문수량,박스수량,판매액
0,2025-02-21,6박스(6개월분)_35%,22,132,3432000
1,2025-02-21,4박스(4개월분)_28%,9,36,1036800
2,2025-02-21,12박스(12개월분)_45%,4,48,1056000
3,2025-02-22,6박스(6개월분)_35%,10,60,1560000
4,2025-02-22,4박스(4개월분)_28%,4,16,460800
5,2025-02-22,12박스(12개월분)_45%,3,36,792000
6,2025-02-23,6박스(6개월분)_35%,11,66,1716000
7,2025-02-23,12박스(12개월분)_45%,4,48,1056000
8,2025-02-23,4박스(4개월분)_28%,4,16,460800
9,2025-02-24,4박스(4개월분)_28%,6,24,691200


In [37]:
df3 = pd.DataFrame(df2.groupby('주문일')['주문수량'].sum().reset_index())
df3['판매액'] = df2.groupby('주문일')['판매액'].sum().values
df3['박스수량'] = df2.groupby('주문일')['박스수량'].sum().values
df3

,주문일,주문수량,판매액,박스수량
0,2025-02-21,35,5524800,216
1,2025-02-22,17,2812800,112
2,2025-02-23,19,3232800,130
3,2025-02-24,15,2419200,96
4,2025-02-25,4,583200,22
5,2025-02-26,4,799200,34
6,2025-02-27,5,806400,32


In [39]:
df3['판매액'].sum()

16178400

In [41]:
from datetime import datetime
now = datetime.now()
now = now.strftime('%Y-%m-%d일 %H시 %M분')

file_path = f'/Users/seanyu_mac/Downloads/공동구매 매출 통계_{now}.xlsx'

with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    df2.to_excel(writer, sheet_name='Sheet1', index=False)
    df3.to_excel(writer, sheet_name='Sheet2', index=False)